<a href="https://colab.research.google.com/github/pksX01/sarvam-llm-finetuning/blob/main/sarvam_2b_llm_finetuning_bhojpuri_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q bitsandbytes transformers peft accelerate datasets trl

In [ ]:
from datasets import load_dataset

ds = load_dataset("SatyamDev/alpaca_data_cleaned_bhojpuri")

In [ ]:
type(ds)

datasets.dataset_dict.DatasetDict

In [ ]:
ds['train'][0]

{'instruction': 'स्वस्थ रहे खातिर तीन गो टिप्स दीं।',
 'input': None,
 'output': '1. संतुलित अवुरी पौष्टिक आहार खाईं: सुनिश्चित करीं कि आपके भोजन में कई प्रकार के फल अवुरी सब्जी, दुबला प्रोटीन, साबुत अनाज अवुरी स्वस्थ वसा शामिल होखे। एहसे आपके शरीर के सबसे निमन तरीका से काम करे खाती जरूरी पोषक तत्व मिले में मदद मिलेला अवुरी पुरान बेमारी से बचाव में मदद मिल सकता।\n\n2. नियमित शारीरिक गतिविधि में शामिल होखे के चाही: हड्डी, मांसपेशी अवुरी हृदय संबंधी स्वास्थ्य के मजबूत बनावे खाती व्यायाम बहुत जरूरी बा। हर हफ्ता कम से कम 150 मिनट मध्यम एरोबिक व्यायाम भा 75 मिनट के जोरदार व्यायाम के लक्ष्य राखीं.\n\n3. पर्याप्त नींद लेवे के बा : शारीरिक अवुरी मानसिक भलाई खाती पर्याप्त गुणवत्ता वाला नींद बहुत जरूरी बा। इ मूड के नियंत्रित करे में मदद करेला, संज्ञानात्मक कामकाज में सुधार करेला अवुरी स्वस्थ विकास अवुरी प्रतिरक्षा के कामकाज के समर्थन करेला। हर रात 7-9 घंटा नींद के लक्ष्य राखीं।'}

In [ ]:
type(ds['train'])

datasets.arrow_dataset.Dataset

In [ ]:
len(ds['train'])

51760

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d0ba57b8-b201-de14-1e3c-29f8f0d28e35)


In [ ]:
def merge_columns(row):
    data = list()
    data.append({"content": row["instruction"], "role": "user"})
    data.append({"content": row["output"], "role": "assistant"})
    row["messages"] = data
    #row['instr_op'] = row['instruction'] + " -> " + str(row['output'])
    return row

In [ ]:
ds['train'] = ds['train'].map(merge_columns)

In [ ]:
ds['train'][0]

{'instruction': 'स्वस्थ रहे खातिर तीन गो टिप्स दीं।',
 'input': None,
 'output': '1. संतुलित अवुरी पौष्टिक आहार खाईं: सुनिश्चित करीं कि आपके भोजन में कई प्रकार के फल अवुरी सब्जी, दुबला प्रोटीन, साबुत अनाज अवुरी स्वस्थ वसा शामिल होखे। एहसे आपके शरीर के सबसे निमन तरीका से काम करे खाती जरूरी पोषक तत्व मिले में मदद मिलेला अवुरी पुरान बेमारी से बचाव में मदद मिल सकता।\n\n2. नियमित शारीरिक गतिविधि में शामिल होखे के चाही: हड्डी, मांसपेशी अवुरी हृदय संबंधी स्वास्थ्य के मजबूत बनावे खाती व्यायाम बहुत जरूरी बा। हर हफ्ता कम से कम 150 मिनट मध्यम एरोबिक व्यायाम भा 75 मिनट के जोरदार व्यायाम के लक्ष्य राखीं.\n\n3. पर्याप्त नींद लेवे के बा : शारीरिक अवुरी मानसिक भलाई खाती पर्याप्त गुणवत्ता वाला नींद बहुत जरूरी बा। इ मूड के नियंत्रित करे में मदद करेला, संज्ञानात्मक कामकाज में सुधार करेला अवुरी स्वस्थ विकास अवुरी प्रतिरक्षा के कामकाज के समर्थन करेला। हर रात 7-9 घंटा नींद के लक्ष्य राखीं।',
 'messages': [{'content': 'स्वस्थ रहे खातिर तीन गो टिप्स दीं।',
   'role': 'user'},
  {'content': '1. संतुलित अवुरी प

In [ ]:
ds['train']['messages'][:3]

[[{'content': 'स्वस्थ रहे खातिर तीन गो टिप्स दीं।', 'role': 'user'},
  {'content': '1. संतुलित अवुरी पौष्टिक आहार खाईं: सुनिश्चित करीं कि आपके भोजन में कई प्रकार के फल अवुरी सब्जी, दुबला प्रोटीन, साबुत अनाज अवुरी स्वस्थ वसा शामिल होखे। एहसे आपके शरीर के सबसे निमन तरीका से काम करे खाती जरूरी पोषक तत्व मिले में मदद मिलेला अवुरी पुरान बेमारी से बचाव में मदद मिल सकता।\n\n2. नियमित शारीरिक गतिविधि में शामिल होखे के चाही: हड्डी, मांसपेशी अवुरी हृदय संबंधी स्वास्थ्य के मजबूत बनावे खाती व्यायाम बहुत जरूरी बा। हर हफ्ता कम से कम 150 मिनट मध्यम एरोबिक व्यायाम भा 75 मिनट के जोरदार व्यायाम के लक्ष्य राखीं.\n\n3. पर्याप्त नींद लेवे के बा : शारीरिक अवुरी मानसिक भलाई खाती पर्याप्त गुणवत्ता वाला नींद बहुत जरूरी बा। इ मूड के नियंत्रित करे में मदद करेला, संज्ञानात्मक कामकाज में सुधार करेला अवुरी स्वस्थ विकास अवुरी प्रतिरक्षा के कामकाज के समर्थन करेला। हर रात 7-9 घंटा नींद के लक्ष्य राखीं।',
   'role': 'assistant'}],
 [{'content': 'तीनों प्राथमिक रंग का हवें?', 'role': 'user'},
  {'content': 'तीनों प्राथम

In [ ]:
ds['train']['messages'][0][0].get('content')

'स्वस्थ रहे खातिर तीन गो टिप्स दीं।'

In [ ]:
#ds = ds.map(lambda row: tokenizer(row['message']), batched=True)

In [ ]:
import torch
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextGenerationPipeline

model_id = "sarvamai/sarvam-2b-v0.5"

'''bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)'''

#model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto')
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# add a chat template to the tokenizer so that it can handle multi-turn conversations
tokenizer.chat_template = """{% if messages[0]['role'] == 'system' %}
                        {% set loop_messages = messages[1:] %}
                        {% set system_message = messages[0]['content'] %}
                        {% else %}
                        {% set loop_messages = messages %}
                        {% set system_message = false %}
                        {% endif %}
                        {% for message in loop_messages %}
                        {% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}
                        {{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}
                        {% endif %}
                        {% if loop.index0 == 0 and system_message != false %}
                        {% set content = '<<SYS>>\\n' + system_message + '\\n<</SYS>>\\n\\n' + message['content'] %}
                        {% else %}
                        {% set content = message['content'] %}
                        {% endif %}
                        {% if message['role'] == 'user' %}
                        {{ bos_token + '[INST] ' + content.strip() + ' [/INST]' }}
                        {% elif message['role'] == 'assistant' %}
                        {{ ' '  + content.strip() + ' ' + eos_token }}
                        {% endif %}
                        {% endfor %}"""

tokenizer.add_tokens("[PAD]", special_tokens=True)
tokenizer.pad_token = "[PAD]"
model.resize_token_embeddings(len(tokenizer))

Embedding(64129, 2048)

In [ ]:
'''from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)'''

'from peft import prepare_model_for_kbit_training\n\nmodel.gradient_checkpointing_enable()\nmodel = prepare_model_for_kbit_training(model)'

In [ ]:
'''def print_trainable_parameters(model):
    all_params = 0
    trainable_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print("Total parameters = ", all_params)
    print("Trainable parameters = ", trainable_params)
    print("% trainable parameters = ", (100 * trainable_params / all_params))'''

'def print_trainable_parameters(model):\n    all_params = 0\n    trainable_params = 0\n    for _, param in model.named_parameters():\n        all_params += param.numel()\n        if param.requires_grad:\n            trainable_params += param.numel()\n    print("Total parameters = ", all_params)\n    print("Trainable parameters = ", trainable_params)\n    print("% trainable parameters = ", (100 * trainable_params / all_params))'

In [ ]:
"""def preprocess_function(example):
    #if example is None or example["messages"] is None or example["messages"][0].get('content') is None or example["messages"][1].get('content') is None:
        #return None
    #if example.get("messages") is None or example.get("messages")[0].get("content") is None or example.get("messages")[1].get("content") is None:
        #return None
    model_ips = tokenizer.apply_chat_template(example["messages"], tokenize=False)
    tokenized_ips = tokenizer(model_ips)
    tokenized_ips["labels"] = tokenized_ips["input_ids"].copy()
    return tokenized_ips"""

def preprocess_function(example):
    if not example.get("messages"):
        return None

    # Ensure all messages have 'content' and 'role', and content is a string
    valid_messages = []
    for msg in example["messages"]:
        if isinstance(msg.get("content"), str) and msg.get("role"):
            valid_messages.append(msg)
        else:
            print(f"Skipping invalid message: {msg}")

    if not valid_messages:
        return None

    try:
        model_ips = tokenizer.apply_chat_template(valid_messages, tokenize=False)
        tokenized_ips = tokenizer(model_ips)
        tokenized_ips["labels"] = tokenized_ips["input_ids"].copy()
        return tokenized_ips
    except Exception as e:
        print(f"Error processing example: {e}")
        return None

In [ ]:
ds = ds.map(preprocess_function, remove_columns=ds["train"].column_names)

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 51760
    })
})

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    lora_dropout=0.0,
    target_modules=["lm_head", "k_proj", "q_proj", "v_proj" "o_proj", "gate_proj", "down_proj", "up_proj"]
    #bias="none",
    #task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 87,269,440 || all params: 2,596,108,352 || trainable%: 3.3615


In [ ]:
#from trl import SFTTrainer
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=ds['train'],
    #dataset_text_field="instr_op",
    args=transformers.TrainingArguments(
        num_train_epochs=1,
        save_total_limit=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
        warmup_steps=10,
        weight_decay=0.0001,
        max_steps=500,
        learning_rate=1e-5,
        #fp16=True,
        bf16=True,
        #save_steps=10,
        logging_steps=50,
        output_dir="sarvam-2b-ft_bhojpuri"
    ),
    data_collator=transformers.DataCollatorForSeq2Seq(tokenizer)
)

#model.config.use_cache = False #enable it for inference
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Step,Training Loss
50,3.806300
100,3.223200
150,2.950400
200,2.726900
250,2.663200
300,2.555600
350,2.447700
400,2.431400
450,2.382400
500,2.398800


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=500, training_loss=2.7585988311767577, metrics={'train_runtime': 5469.44, 'train_samples_per_second': 0.366, 'train_steps_per_second': 0.091, 'total_flos': 1.08486453667968e+16, 'train_loss': 2.7585988311767577, 'epoch': 0.03863987635239567})

In [ ]:
"""model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_to_save.save_pretrained("outputs")"""

'model_to_save = trainer.model.module if hasattr(trainer.model, \'module\') else trainer.model\nmodel_to_save.save_pretrained("outputs")'

In [ ]:
"""lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)"""

"lora_config = LoraConfig.from_pretrained('outputs')\nmodel = get_peft_model(model, lora_config)"

In [ ]:
"""question = input()

ips = tokenizer(question, return_tensors="pt").to("cuda:0")
ops = model.generate(**ips, max_new_tokens=50)
print(tokenizer.decode(ops[0], skip_special_tokens=True))"""

'question = input()\n\nips = tokenizer(question, return_tensors="pt").to("cuda:0")\nops = model.generate(**ips, max_new_tokens=50)\nprint(tokenizer.decode(ops[0], skip_special_tokens=True))'

In [ ]:
def generate_response(question):
    message = [{"role": "user", "content": question}]
    model_ip = tokenizer.apply_chat_template(message, tokenize=False)
    tokenized_ip = tokenizer(model_ip, return_tensors="pt").to("cuda")
    model.eval()
    op_tokens = model.generate(
        **tokenized_ip,
        max_new_tokens=250,
        temperature=0.01,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    op = tokenizer.decode(op_tokens[0], skip_special_tokens=True)
    return op

In [ ]:
generate_response("भारत के पहिला प्रधानमंत्री के रहे?")

'                       [INST] भारत के पहिला प्रधानमंत्री के रहे? [/INST]\n                         1947 में आजादी मिलने के बाद, जवाहरलाल नेहरु भारत के पहले प्रधान मंत्री रहलें। एह व्यक्ति का जन्म कलकत्ता (अब कोलकाता) में हुआह जेईं कि जब ब्रिटिश शासन चलत बा. एह लोगन के नाम से प्रसिद्ध होला जवना कि एगो "आधुनिक भारतीय इतिहास के जनक" आ "भारत के पितामह"। एह नेता के नेतृत्व में भारत स्वतंत्रता के दौर में आगे बढ़ा, और देश को एक स्वतंत्र राष्ट्र बनावल जा सकी। </s>\n'

In [ ]:
generate_response("स्वस्थ रहे खातिर तीन गो टिप्स दीं।")

'                       [INST] स्वस्थ रहे खातिर तीन गो टिप्स दीं। [/INST]\n                         1. भरपूर पानी पिएं: शरीर के ठीक से काम करे खातिर रोजाना कमतहल से आठ गिलास पानी पीवे जरूरी बाड़ें।\n2. संतुलित आहार लेवेः फल, सब्जियां, साबुत अनाज, लीन प्रोटीन आ दुग्ध उत्पाद शामिल वाला पौष्टिक भोजन खावे में मदद मिले ला।\n3. नियमित व्यायाम करीं: शारीरिक गतिविधि सेहतमंद रहले खातील, खासकर हड्डियाँ मजबूत कइलनी खातिर वजन उठाना, चलना, दौड़ना या योगासन करीं। '

In [ ]:
generate_response("हमनी के वायु प्रदूषण के कइसे कम कर सकेनी जा?")

'                       [INST] हमनी के वायु प्रदूषण के कइसे कम कर सकेनी जा? [/INST]\n                          कई तरीके से हवा में प्रदूषक पदार्थ के मात्रा कम होला जा सकता:\n1. ऊर्जा संरक्षण आ बिजली उत्पादन के वैकल्पिक स्रोत इस्तेमाल करे खातिर प्रोत्साहित करीं जवना कि जीवाश्म ईंधन पर निर्भरता कम होखे।\n2. पेड़ लगावे, खासकर शहरी क्षेत्र में जहाँ हवेली के संख्या बढ़ावे ला रहे बाड़ें। पेड़ कार्बन डाइऑक्साइड अवशोषित करत रहलें आ ऑक्सीजन छोड़ेले, जिससे वायु गुणवत्ता सुधार सके।\n3. सार्वजनिक परिवहन का उपयोग बढ़ावे, कारपूलिंग या बाइक चलावे के प्रोत्साहन देवे ताकि सड़क पर गाड़ियों की संख्या कम हो सके।\n4. औद्योगिक उत्सर्जन के नियमन के कइल जा सके, जैसे कि कारखाने के पास धुआँ-छंटाई उपकरण लागू करे खातीं कि हानिकारक रसायनों के रिलीज कम हो सके।\n5. कचरा निपटान के उचित तरीका अपनावे खातिर प्रेरित करीं, कचरे को ठीक से अलग करे खातिर और पुनर्चक्रण के बढ़ावा दूखीं।\n6. पर्यावरण अनुकूल उत्पाद खरीद लेवे खातिर प्रोत्साहित करीं, जोकि प्राकृतिक सामग्री से बना होखे आ न्यूनतम पर्यावरणीय प्रभाव वाला होखे।\n7. जाग

In [ ]:
generate_response("october k mahine me bharat me kaha kaha ghumal thik rahi?")

'                       [INST] october k mahine me bharat me kaha kaha ghumal thik rahi? [/INST]\n अक्टूबर के महीने में भारत के कई शहर और कस्बे घूमने लायक होलें। कुछ लोकप्रिय गंतव्य स्थल हैं:\n- मुंबई (मुंबई) - गोवा, महाराष्ट्र\n- दिल्ली - जयपुर, राजस्थान\n- कोलकाता (कोलकाता) - चेन्नई, तमिलनाडु\n- हैदराबाद (हैदराबाद) - बैंगलोर, कर्नाटक\n- पुणे (पुणे) - अहमदाबाद, गुजरात\n- बेंगलुरु (बैंगलूरू) - इंदौर, मध्य प्रदेश\n- नागपुर (नागपूर) - भोपाल, मध्यप्रदेश\n- लखनऊ (आलाहाबाद) - रायपुर, छत्तीसगढ़\n- वाराणसी (वाराणसी) - देहरादून, उत्तराखंड\n- अमृतसर (अमृतसर) - भुवनेश्वर, ओडिशा\n- चंडीगढ़ (चंडीगढ़) - पटना, बिहार\n- आगरा (फतेहपुर सीकरी) - बड़ौदा, राजस्थान\n- कानपुर (काननपुर) - उज्जैन, मध्य प्रदेश\n- इंदौर (इंदौर) - खजुराहो, मध्य प्रदेश\n- जालंधर (जोधपुर) - उदयपुर, राजस्थान\n- लुधियाना (लुधियाना) - उदयपुरा, पंजाब\n'

In [ ]:
generate_response("हमार सवाल के जवाब भोजपुरी भाषा में दीं। इज़राइल देश के स्थापना कब भइल रहे और ओमे में केकर केकर महत्वपूर्ण भूमिका रहे?")

'                       [INST] हमार सवाल के जवाब भोजपुरी भाषा में दीं। इज़राइल देश के स्थापना कब भइल रहे और ओमे में केकर केकर महत्वपूर्ण भूमिका रहे? [/INST]\n                         1948 ईगो. एह तारीख से इजरायल राज्य बनाल बा जवना में कई जातीय समूह शामिल होला, जिनमें यहूदी लोग भी शामिल जेहें हकीकत में अरब नागरिक थे। एहराक में मुख्य रूप से यहूदियों आ कुछ ईसाई समुदाय के निवास स्थान बन गइल बा जब तक कि 20वीं सदी के मध्य में फिलिस्तीनी-अरबों के बीच संघर्ष शुरू न हो जाईं। इन संघर्षों के परिणामस्वरूप अंततः 1967 में छह दिवसीय युद्ध के बाद इसराइल ने वेस्ट बैंक पर कब्जा कर लिया। तब से, इज़राइली सरकार लगातार कब्जे वाले क्षेत्रों में बड़त जारी रखेले रहल बा, जिसमें पूर्वी यरुशलम शहर भी शामिल है। '

Pushing fine-tuned model to HuggingFace Hub